In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor, Lambda, Resize, Normalize
from torch.utils.data import DataLoader
import pandas as pd

from transformers import ViTForImageClassification, TrainingArguments, Trainer

In [ ]:
DIRECTROY = 'data'
MODEL_PATH = 'models'
BATCH_SIZE = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
df_train = pd.read_csv(f'{DIRECTROY}/train.csv') 
num_classes = len(df_train['class'].unique())

In [ ]:
image_transforms = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

data = ImageFolder(f'{DIRECTROY}/train', transform=image_transforms)
train_dataset = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=num_classes)
model = model.to(device)
model.parameters()